In [1]:
#this is array and table and machine learning stuff
import math
import numpy as np
import pandas as pd
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#this is xgboost stuff
from xgboost import XGBClassifier
from xgboost import XGBModel
from xgboost import XGBRegressor

#this is catboost
from catboost import CatBoostClassifier, Pool
import ipywidgets
import IPython

#this is plot stuff
import matplotlib.pyplot as plt
import seaborn as sns

#idk what this is but Data100 used it
import scipy.stats

/var/folders/78/wrm63b0n5jxg2yyw_6wf_66h0000gn/T/ipykernel_896/1744575342.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Gotta Add Our Table First!

apples

In [2]:
url = 'apple_quality.csv'
apple = pd.read_csv(url)

In [3]:
apple

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good
...,...,...,...,...,...,...,...,...,...
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806,good
3999,3999.0,0.278540,-1.715505,0.121217,-1.154075,1.266677,-0.776571,1.599796456,good


Next, we need to clean our data to use it in a model. We remove all rows with NaN values. This only removes the last row, but for other sets of data, it might be a bit more.

Afterwards, since we're going to attempt to predict whether an apple is good or bad, we put all variables in the X and the good and bad are our Y. For models, categorical data can be put into binary form, so our models can use numerical values instead. Since we only have 2 outcomes (good or bad), we can just use 0 and 1. 

In [4]:
#cleaned the data to ensure it can be used in a model.
apple = apple.dropna()
apple = apple.replace({"good":1, "bad":0})
X = apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight"]]
Y = apple['Quality']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=42)

/var/folders/78/wrm63b0n5jxg2yyw_6wf_66h0000gn/T/ipykernel_896/3367397462.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  apple = apple.replace({"good":1, "bad":0})


Let's try using XGB first!

In [5]:
model = XGBClassifier()
model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:
predicted_values = model.predict(X_test)
accuracy = accuracy_score(Y_test, predicted_values)
accuracy

0.8875

In [7]:
print("My accuracy is", accuracy, ".")
xgb_percent_accuracy = 100*accuracy
print("My accuracy is", xgb_percent_accuracy, "%!")

My accuracy is 0.8875 .
My accuracy is 88.75 %!


What if we use other models? Let's try logistic regression.

In [8]:
model = lm.LogisticRegression()
model.fit(X_train, Y_train)
log_accuracy = model.score(X_test, Y_test)
log_accuracy

0.7575

In [9]:
print("The logistic model has an accuracy of 75.75%, while the XGBoost model had an accuracy of 88.75%!")

The logistic model has an accuracy of 75.75%, while the XGBoost model had an accuracy of 88.75%!


When I tried to use Acidity in the XBG model earlier, the model said it was the wrong type of data, so I disregarded it. Let's see if using Acidity increases our accuracy!

In [10]:
apple = apple.dropna()
apple = apple.replace({"good":1, "bad":0})
X = apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight", "Acidity"]]
Y = apple['Quality']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=42)

In [11]:
acid_model = lm.LogisticRegression()
acid_model.fit(X_train, Y_train)
log_accuracy_acid = acid_model.score(X_test, Y_test)
log_accuracy_acid

0.75

Adding acidity decreases the accuracy ironically. Let's try checking the coefficients?

In [14]:
model.coef_

array([[ 0.3478219 ,  0.54830137, -0.09688376, -0.03638919,  0.50190931,
         0.2322198 ]])

In [20]:
print(['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight"])
print(model.coef_)

['Juiciness', 'Size', 'Ripeness', 'Crunchiness', 'Sweetness', 'Weight']
[[ 0.3478219   0.54830137 -0.09688376 -0.03638919  0.50190931  0.2322198 ]]


In [38]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

sigmoid( np.dot(apple[['Juiciness', 'Size', "Ripeness", "Crunchiness", "Sweetness", "Weight"]], model.coef_) + model.intercept_)

ValueError: shapes (4000,6) and (1,6) not aligned: 6 (dim 1) != 1 (dim 0)

In [21]:
acid_model.coef_

array([[ 0.43923019,  0.64646856, -0.12407409,  0.01245805,  0.57010037,
         0.2663912 , -0.29894365]])

Are there any other models we could potentially use?

In [22]:
catboost_train_data = Pool(X_train, Y_train)
catboost_model = CatBoostClassifier()

In [23]:
catboost_model.fit(catboost_train_data, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.017802
0:	learn: 0.6821551	total: 67.1ms	remaining: 1m 7s
1:	learn: 0.6726690	total: 71ms	remaining: 35.4s
2:	learn: 0.6645269	total: 73.8ms	remaining: 24.5s
3:	learn: 0.6562081	total: 76.2ms	remaining: 19s
4:	learn: 0.6476392	total: 78.3ms	remaining: 15.6s
5:	learn: 0.6400178	total: 80.6ms	remaining: 13.4s
6:	learn: 0.6319314	total: 82.8ms	remaining: 11.7s
7:	learn: 0.6239141	total: 85.4ms	remaining: 10.6s
8:	learn: 0.6160814	total: 87.6ms	remaining: 9.64s
9:	learn: 0.6085779	total: 90.6ms	remaining: 8.97s
10:	learn: 0.6023016	total: 92.7ms	remaining: 8.33s
11:	learn: 0.5975043	total: 94.8ms	remaining: 7.8s
12:	learn: 0.5908434	total: 102ms	remaining: 7.72s
13:	learn: 0.5863855	total: 104ms	remaining: 7.32s
14:	learn: 0.5815882	total: 106ms	remaining: 6.97s
15:	learn: 0.5769304	total: 108ms	remaining: 6.67s
16:	learn: 0.5716923	total: 111ms	remaining: 6.42s
17:	learn: 0.5657041	total: 115ms	remaining: 6.27s
18:	learn: 0.5608243	total: 118ms	remaining: 6.08s
19:	

In [24]:
catboost_model.score(catboost_train_data)

0.97

In [25]:
catboost_test_data = Pool(X_test)

In [26]:
catboost_model.predict(catboost_test_data)

array([1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,

In [27]:
catboost_score = catboost_model.score(X_test, Y_test)
catboost_score

0.92

Catboost has the highest accuracy!

To summarize...

In [28]:
catboost_percent = catboost_score*100
xgb_percent_accuracy = 100*accuracy
log_percent = log_accuracy*100
print("Catboost has the highest score of", catboost_percent, "%.")
print("XGB has an accuracy of", xgb_percent_accuracy, "%.")
print("The logistic regression has the lowest accuracy, an accuracy of", log_percent, "%.")

Catboost has the highest score of 92.0 %.
XGB has an accuracy of 88.75 %.
The logistic regression has the lowest accuracy, an accuracy of 75.75 %.


In [29]:
# use coef to discover the weight of features for logistic regression

In [30]:
# regularize logistic regression to make it more accurate? regularization helps decrease overfitting?


In [31]:
# training loss research